# * Adhoc : Target

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 18)

## ETL Process...

### Step 1 : Import Data Source (Excel)
    Read Excel data

In [54]:
''' File Config '''

src_file = 'C:\Ruz\Pentaho\Jobs\Input\Target\Source\Ref-K.Voraphon\DTAC_Sales_Target\Target_DTAC_Sales_Y2024.xlsx'
src_sheet = 'Rawdata'

# Create DataFrame
src_df = pd.read_excel(src_file, sheet_name=src_sheet, index_col=None) 
src_df['MONTH_KEY'] = src_df['DATA_MONTH'].str.upper()
# src_df = src_df.replace(np.nan, None)
src_df.tail(3)

,DATA_DATE,DATA_MONTH,DATA_YEAR,METRIC_CD,METRIC_NAME,METRIC_VALUES,COMP_CD,VERSION,AREA_CD,ARE_DESC,AREA_TYPE,MONTH_KEY
17289,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,9.237334e+07,DTAC,T,84Z,SURAT THANI,H,MAY
17290,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,7.089238e+07,DTAC,T,93X,"TRANG, SATUN, PHATTHALUNG",H,MAY
17291,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY


### Step 2 : Import DIM_TIME (CSV)
    Read CSV data

In [55]:
''' File Config '''

dt_file = '../CFW/data/dim_time.csv'
dt_df = pd.read_csv(dt_file)

# Add KEY column
dt_df['MONTH_KEY'] = dt_df['MONTH_SHORT'].str.upper()
dt_df.tail(3)

,TM_KEY_DAY,DAY_DESC,DAY_SHORT,DAY_NO,DATE_VALUE,DAY_OF_WEEK,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TRUE_WEEK,TM_KEY_WK,...,TM_KEY_QTR,QUARTER_NO,TM_KEY_YR,YEAR_DESC,YEAR_DESC_TH,PERIODFLAG,ETL_DATE,ETL_UPDATE,LOAD_DATE,MONTH_KEY
1093,20251229,Monday,Mon,29,2025-12-29,1,31,2026001,1,2025053,...,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41,DEC
1094,20251230,Tuesday,Tue,30,2025-12-30,2,31,2026001,1,2025053,...,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41,DEC
1095,20251231,Wednesday,Wed,31,2025-12-31,3,31,2026001,1,2025053,...,20254,4,2025,2025,2568,EMQY,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41,DEC


In [56]:
# pd.merge(left, right, how="left", on=["key1", "key2"])

# df1 = pd.merge(src_df, dt_df, how='left', on=['MONTH_KEY'])
df1 = pd.merge(src_df, dt_df[['MONTH_KEY', 'TM_KEY_MTH']], how='left', on='MONTH_KEY')
df1.tail(3)

,DATA_DATE,DATA_MONTH,DATA_YEAR,METRIC_CD,METRIC_NAME,METRIC_VALUES,COMP_CD,VERSION,AREA_CD,ARE_DESC,AREA_TYPE,MONTH_KEY,TM_KEY_MTH
1570203,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505
1570204,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505
1570205,20240523,MAY,2024,DB0R000100,Total Revenue : DTAC,NaN,DTAC,T,NaN,True corp,H,MAY,202505


In [58]:
# x1 = src_df.set_index('MONTH_KEY')
# x2 = dt_df[['MONTH_KEY', 'TM_KEY_MTH']].set_index('MONTH_KEY')

# df2 = x1.join(x2, how='left', on='MONTH_KEY')
# df2.tail(3)